Modified from https://github.com/melanieabrams-pub/RH-seq_with_barcoding/blob/main/annotate_poolfile_for_barseq_v1.2_JSkerker.ipynb and 
https://github.com/melanieabrams-pub/kluyv_RH-seq_with_barcoding/blob/main/annotate_poolfile_kluyv.py

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

### Load GFF annotations

In [2]:
#you don't need to change the path below - this is the most recent annotation file (5/12/22)
gff_file = '/usr2/people/shollyt22/shollyt22/Barseq_July_2023/OORB003_TnSeq/for_analysis/modified_gff_file_finalized_08_08_2023'

In [3]:
gff = pd.read_csv(gff_file, sep='\t')

In [4]:
t = gff['scaffold'].unique()
print(t)

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7']


In [5]:
gff.head()

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
1,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
2,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
3,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335
4,chr1,JGI,CDS,15458.0,25533.0,+,ID=gene_5,proteinId=2121898


In [6]:
gff

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
1,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
2,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
3,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335
4,chr1,JGI,CDS,15458.0,25533.0,+,ID=gene_5,proteinId=2121898
...,...,...,...,...,...,...,...,...
9105,chr7,JGI,CDS,4089438.0,4091337.0,+,ID=gene_10836,proteinId=103797
9106,chr7,JGI,CDS,4095568.0,4097284.0,+,ID=gene_10837,proteinId=2071098
9107,chr7,JGI,CDS,4098122.0,4098907.0,-,ID=gene_10838,proteinId=2114022
9108,chr7,JGI,CDS,4100606.0,4101293.0,-,ID=gene_10840,proteinId=2114023


### Match barcode insert locations to gene annotations

In [7]:
!pwd

/auto/sahara/namib/home/shollyt22/shollyt22/Barseq_July_2023/OORB003_TnSeq/for_analysis


In [8]:
#CHANGE IF NEEDED: load COMBINED poolfile generated in TNseq mapping step

poolfile = '/usr2/people/shollyt22/shollyt22/Barseq_July_2023/OORB003_TnSeq/for_analysis/TNSeq_mapping_output_new_genome_without_vector/Tnseq_08_10_2023_without_vector_seq_poolfile'

#CHANGE IF NEEDED: annotated pool file path to output

annotated_poolfile_path = '/usr2/people/shollyt22/shollyt22/Barseq_July_2023/OORB003_TnSeq/for_analysis/TNSeq_mapping_output_new_genome_without_vector/Tnseq_08_10_2023_annotated.csv'

In [9]:
df = pd.read_csv(poolfile,sep='\t')
df.head()

,barcode,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
0,AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]","{'Null:0:+': 0, 'insert:7:+': 1, 'plasmid:17:+': 1}"
1,CTAGGTTGGAAGAACACACT,AGTGTGTTCTTCCAACCTAG,2181,2181,chr2,-,1265125,Concat,1408,639,"[('chr2:1265125:-', 1408), ('chr2:1265122:-', 4), ('chr2:2592533:+', 4), ('chr4:2180439:-', 3), ('chr1:8295228:+', 2), ('chr1:761707:+', 2), ('chr1:212137:+', 2), ('chr1:4996653:+', 2), ('chr2:4327814:-', 2), ('chr2:2395934:-', 2), ('chr5:290815:+', 2), ('chr2:4314401:+', 2), ('chr2:1105155:+', 2), ('chr2:4661375:+', 2), ('chr2:1265123:-', 2), ('chr1:954628:+', 1), ('chr7:2785623:+', 1), ('chr6:1720330:-', 1), ('chr1:2266535:-', 1), ('chr1:7182043:+', 1), ('chr7:1448797:+', 1), ('chr2:4234596:+', 1), ('chr3:3398001:-', 1), ('chr5:4165320:-', 1), ('chr3:4885693:+', 1), ('chr7:1746287:+', 1), ('chr1:9216225:+', 1), ('chr2:1114508:+', 1), ('chr6:3039724:-', 1), ('chr3:4887708:-', 1), ('chr1:7436906:+', 1), ('chr1:10013646:+', 1), ('chr2:1537912:+', 1), ('chr2:561950:+', 1), ('chr4:1614007:-', 1), ('chr1:7748225:-', 1), ('chr2:2194574:-', 1), ('chr2:1553014:+', 1), ('chr6:2863705:-', 1), ('chr2:664147:+', 1), ('chr2:2256078:+', 1), ('chr1:2700577:-', 1), ('chr1:10103069:+', 1), ('chr1:6914244:-', 1), ('chr5:3062535:-', 1), ('chr1:2068221:+', 1), ('chr2:5250247:-', 1), ('chr2:5426773:+', 1), ('chr6:288713:+', 1), ('chr1:4328966:+', 1), ('chr6:612013:+', 1), ('chr6:4085274:+', 1), ('chr3:1944078:+', 1), ('chr2:1265129:-', 1), ('chr1:7972497:-', 1), ('chr2:1855143:+', 1), ('chr2:4291019:-', 1), ('chr2:2305298:+', 1), ('chr5:2709153:+', 1), ('chr2:1114472:+', 1), ('chr5:2602384:+', 1), ('chr6:2322131:-', 1), ('chr4:1625473:+', 1), ('chr2:2720688:-', 1), ('chr2:470383:-', 1), ('chr2:1393197:+', 1), ('chr7:3977511:-', 1), ('chr2:4258145:-', 1), ('chr3:1646949:+', 1), ('chr1:242927:-', 1), ('chr1:2295113:-', 1), ('chr1:6511163:-', 1), ('chr1:8696473:-', 1), ('chr1:6354388:-', 1), ('chr1:2643374:+', 1), ('chr1:8732382:-', 1), ('chr1:6886045:+', 1), ('chr7:40481:-', 1), ('chr4:144100:+', 1), ('chr6:72928:-', 1), ('chr5:2355634:+', 1), ('chr1:8031958:-', 1), ('chr1:212113:-', 1), ('chr2:1137525:+', 1), ('chr1:6138485:+', 1), ('chr2:4663687:-', 1), ('chr6:105262:-', 1), ('chr6:1914656:-', 1), ('chr7:2979024:-', 1), ('chr7:2372256:-', 1), ('chr2:1265085:+', 1), ('chr6:2608888:-', 1), ('chr2:291854:+', 1), ('chr7:2325040:-', 1), ('chr7:1120573:+', 1), ('chr3:2416675:+', 1), ('chr1:7836540:-', 1), ('chr3:1904761:+', 1), ('chr1:182855:-', 1), ('chr1:1449141:+', 1), ('chr3:1722190:+', 1), ('chr6:3148935:-', 1), ('chr1:7250446:-', 1), ('chr1:5513126:+', 1), ('chr2:455150:+', 1), ('chr2:1866159:+', 1), ('chr3:291875:+', 1), ('chr1:3821780:-', 1), ('chr3:2107619:-', 1), ('chr2:4605968:-', 1), ('chr5:3773389:-', 1), ('chr1:10033758:-', 1), ('chr7:1096722:-', 1), ('chr2:1154399:-', 1), ('chr3:626243:+', 1), ('chr1:845857:-', 1)]","{'Null:0:+': 0, 'insert:1877:+': 23, 'insert:7:+': 551, 'plasmid:1885:-': 1, 'insert:4:+': 1, 'insert:12:+': 1, 'insert:9:+': 2, 'plasmid:302:-': 1, 'plasmid:5572:-': 1, 'insert:1580:+': 1, 'insert:1:+': 10, 'insert:5:+': 1, 'insert:1456:-': 1, 'insert:1748:-': 1, 'insert:10:+': 1, 'insert:8:+': 1, 'insert:936:-': 1, 'insert:438:+': 1, 'plasmid:1983:+': 2, 'insert:21:+': 1, 'plasmid:252:+': 2, 'insert:890:+': 2, 'insert:13:+': 4, 'insert:1689:+': 1, 'insert:6:+': 1, 'insert:533:+': 1, 'insert:68:+': 1, 'plasmid:17:+': 1, 'insert:24:+': 1, 'insert:222:+': 1, 'insert:3:+': 2, 'insert:1607:-': 2, 'insert:1724:+': 1, 'insert:1880:-': 1, 'insert:213:-': 1, 'insert:1889:+': 1, 'insert:108:+': 2, 'plasmid:5332:+': 1, 'insert:478:+': 1, 'insert:844:+': 1, 'insert:180:+': 1, 'plasmid:1216:+': 1, 'insert:30:+': 1, 'plasmid:5596:-': 3, 'insert:1701:-': 1, 'insert:1004:+': 1, 'insert:14:+': 1}"
2,TAGCCTAATTCTCTTTTAAT,

Check that each barcode in the dataframe is unique (the two numbers below should be the same):

In [10]:
print('Number of insertions in poolfile: {}'.format(df.shape[0]))
print('Number of unique barcodes in poolfile: {}'.format(len(df['barcode'].unique())))

Number of insertions in poolfile: 4986
Number of unique barcodes in poolfile: 4986


In [11]:
#df = df.set_index('transposonLocation') #This is what was there and it is showing error. The transposonlocation was changed to barcode in the next line
df = df.set_index('barcode')

In [12]:
df.columns

Index(['rcbarcode', 'nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
       'nMainLocation', 'nInsert', 'All genomic mappings',
       'All insert mappings'],
      dtype='object')

In [13]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
barcode,,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]","{'Null:0:+': 0, 'insert:7:+': 1, 'plasmid:17:+': 1}"
CTAGGTTGGAAGAACACACT,AGTGTGTTCTTCCAACCTAG,2181,2181,chr2,-,1265125,Concat,1408,639,"[('chr2:1265125:-', 1408), ('chr2:1265122:-', 4), ('chr2:2592533:+', 4), ('chr4:2180439:-', 3), ('chr1:8295228:+', 2), ('chr1:761707:+', 2), ('chr1:212137:+', 2), ('chr1:4996653:+', 2), ('chr2:4327814:-', 2), ('chr2:2395934:-', 2), ('chr5:290815:+', 2), ('chr2:4314401:+', 2), ('chr2:1105155:+', 2), ('chr2:4661375:+', 2), ('chr2:1265123:-', 2), ('chr1:954628:+', 1), ('chr7:2785623:+', 1), ('chr6:1720330:-', 1), ('chr1:2266535:-', 1), ('chr1:7182043:+', 1), ('chr7:1448797:+', 1), ('chr2:4234596:+', 1), ('chr3:3398001:-', 1), ('chr5:4165320:-', 1), ('chr3:4885693:+', 1), ('chr7:1746287:+', 1), ('chr1:9216225:+', 1), ('chr2:1114508:+', 1), ('chr6:3039724:-', 1), ('chr3:4887708:-', 1), ('chr1:7436906:+', 1), ('chr1:10013646:+', 1), ('chr2:1537912:+', 1), ('chr2:561950:+', 1), ('chr4:1614007:-', 1), ('chr1:7748225:-', 1), ('chr2:2194574:-', 1), ('chr2:1553014:+', 1), ('chr6:2863705:-', 1), ('chr2:664147:+', 1), ('chr2:2256078:+', 1), ('chr1:2700577:-', 1), ('chr1:10103069:+', 1), ('chr1:6914244:-', 1), ('chr5:3062535:-', 1), ('chr1:2068221:+', 1), ('chr2:5250247:-', 1), ('chr2:5426773:+', 1), ('chr6:288713:+', 1), ('chr1:4328966:+', 1), ('chr6:612013:+', 1), ('chr6:4085274:+', 1), ('chr3:1944078:+', 1), ('chr2:1265129:-', 1), ('chr1:7972497:-', 1), ('chr2:1855143:+', 1), ('chr2:4291019:-', 1), ('chr2:2305298:+', 1), ('chr5:2709153:+', 1), ('chr2:1114472:+', 1), ('chr5:2602384:+', 1), ('chr6:2322131:-', 1), ('chr4:1625473:+', 1), ('chr2:2720688:-', 1), ('chr2:470383:-', 1), ('chr2:1393197:+', 1), ('chr7:3977511:-', 1), ('chr2:4258145:-', 1), ('chr3:1646949:+', 1), ('chr1:242927:-', 1), ('chr1:2295113:-', 1), ('chr1:6511163:-', 1), ('chr1:8696473:-', 1), ('chr1:6354388:-', 1), ('chr1:2643374:+', 1), ('chr1:8732382:-', 1), ('chr1:6886045:+', 1), ('chr7:40481:-', 1), ('chr4:144100:+', 1), ('chr6:72928:-', 1), ('chr5:2355634:+', 1), ('chr1:8031958:-', 1), ('chr1:212113:-', 1), ('chr2:1137525:+', 1), ('chr1:6138485:+', 1), ('chr2:4663687:-', 1), ('chr6:105262:-', 1), ('chr6:1914656:-', 1), ('chr7:2979024:-', 1), ('chr7:2372256:-', 1), ('chr2:1265085:+', 1), ('chr6:2608888:-', 1), ('chr2:291854:+', 1), ('chr7:2325040:-', 1), ('chr7:1120573:+', 1), ('chr3:2416675:+', 1), ('chr1:7836540:-', 1), ('chr3:1904761:+', 1), ('chr1:182855:-', 1), ('chr1:1449141:+', 1), ('chr3:1722190:+', 1), ('chr6:3148935:-', 1), ('chr1:7250446:-', 1), ('chr1:5513126:+', 1), ('chr2:455150:+', 1), ('chr2:1866159:+', 1), ('chr3:291875:+', 1), ('chr1:3821780:-', 1), ('chr3:2107619:-', 1), ('chr2:4605968:-', 1), ('chr5:3773389:-', 1), ('chr1:10033758:-', 1), ('chr7:1096722:-', 1), ('chr2:1154399:-', 1), ('chr3:626243:+', 1), ('chr1:845857:-', 1)]","{'Null:0:+': 0, 'insert:1877:+': 23, 'insert:7:+': 551, 'plasmid:1885:-': 1, 'insert:4:+': 1, 'insert:12:+': 1, 'insert:9:+': 2, 'plasmid:302:-': 1, 'plasmid:5572:-': 1, 'insert:1580:+': 1, 'insert:1:+': 10, 'insert:5:+': 1, 'insert:1456:-': 1, 'insert:1748:-': 1, 'insert:10:+': 1, 'insert:8:+': 1, 'insert:936:-': 1, 'insert:438:+': 1, 'plasmid:1983:+': 2, 'insert:21:+': 1, 'plasmid:252:+': 2, 'insert:890:+': 2, 'insert:13:+': 4, 'insert:1689:+': 1, 'insert:6:+': 1, 'insert:533:+': 1, 'insert:68:+': 1, 'plasmid:17:+': 1, 'insert:24:+': 1, 'insert:222:+': 1, 'insert:3:+': 2, 'insert:1607:-': 2, 'insert:1724:+': 1, 'insert:1880:-': 1, 'insert:213:-': 1, 'insert:1889:+': 1, 'insert:108:+': 2, 'plasmid:5332:+': 1, 'insert:478:+': 1, 'insert:844:+': 1, 'insert:180:+': 1, 'plasmid:1216:+': 1, 'insert:30:+': 1, 'plasmid:5596:-': 3, 'insert:1701:-': 1, 'insert:1004:+': 1, 'insert:14:+': 1}"
TAGCCTAATTCTCTTT

In [14]:
df=df.drop(df.columns[-1],axis=1)

In [15]:
# df.columns = ['rcbarcode','nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
#        'nMainLocation', 'nInsert', 'All genomic mappings',
#        'All insert mappings']

In [16]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings
barcode,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]"
CTAGGTTGGAAGAACACACT,AGTGTGTTCTTCCAACCTAG,2181,2181,chr2,-,1265125,Concat,1408,639,"[('chr2:1265125:-', 1408), ('chr2:1265122:-', 4), ('chr2:2592533:+', 4), ('chr4:2180439:-', 3), ('chr1:8295228:+', 2), ('chr1:761707:+', 2), ('chr1:212137:+', 2), ('chr1:4996653:+', 2), ('chr2:4327814:-', 2), ('chr2:2395934:-', 2), ('chr5:290815:+', 2), ('chr2:4314401:+', 2), ('chr2:1105155:+', 2), ('chr2:4661375:+', 2), ('chr2:1265123:-', 2), ('chr1:954628:+', 1), ('chr7:2785623:+', 1), ('chr6:1720330:-', 1), ('chr1:2266535:-', 1), ('chr1:7182043:+', 1), ('chr7:1448797:+', 1), ('chr2:4234596:+', 1), ('chr3:3398001:-', 1), ('chr5:4165320:-', 1), ('chr3:4885693:+', 1), ('chr7:1746287:+', 1), ('chr1:9216225:+', 1), ('chr2:1114508:+', 1), ('chr6:3039724:-', 1), ('chr3:4887708:-', 1), ('chr1:7436906:+', 1), ('chr1:10013646:+', 1), ('chr2:1537912:+', 1), ('chr2:561950:+', 1), ('chr4:1614007:-', 1), ('chr1:7748225:-', 1), ('chr2:2194574:-', 1), ('chr2:1553014:+', 1), ('chr6:2863705:-', 1), ('chr2:664147:+', 1), ('chr2:2256078:+', 1), ('chr1:2700577:-', 1), ('chr1:10103069:+', 1), ('chr1:6914244:-', 1), ('chr5:3062535:-', 1), ('chr1:2068221:+', 1), ('chr2:5250247:-', 1), ('chr2:5426773:+', 1), ('chr6:288713:+', 1), ('chr1:4328966:+', 1), ('chr6:612013:+', 1), ('chr6:4085274:+', 1), ('chr3:1944078:+', 1), ('chr2:1265129:-', 1), ('chr1:7972497:-', 1), ('chr2:1855143:+', 1), ('chr2:4291019:-', 1), ('chr2:2305298:+', 1), ('chr5:2709153:+', 1), ('chr2:1114472:+', 1), ('chr5:2602384:+', 1), ('chr6:2322131:-', 1), ('chr4:1625473:+', 1), ('chr2:2720688:-', 1), ('chr2:470383:-', 1), ('chr2:1393197:+', 1), ('chr7:3977511:-', 1), ('chr2:4258145:-', 1), ('chr3:1646949:+', 1), ('chr1:242927:-', 1), ('chr1:2295113:-', 1), ('chr1:6511163:-', 1), ('chr1:8696473:-', 1), ('chr1:6354388:-', 1), ('chr1:2643374:+', 1), ('chr1:8732382:-', 1), ('chr1:6886045:+', 1), ('chr7:40481:-', 1), ('chr4:144100:+', 1), ('chr6:72928:-', 1), ('chr5:2355634:+', 1), ('chr1:8031958:-', 1), ('chr1:212113:-', 1), ('chr2:1137525:+', 1), ('chr1:6138485:+', 1), ('chr2:4663687:-', 1), ('chr6:105262:-', 1), ('chr6:1914656:-', 1), ('chr7:2979024:-', 1), ('chr7:2372256:-', 1), ('chr2:1265085:+', 1), ('chr6:2608888:-', 1), ('chr2:291854:+', 1), ('chr7:2325040:-', 1), ('chr7:1120573:+', 1), ('chr3:2416675:+', 1), ('chr1:7836540:-', 1), ('chr3:1904761:+', 1), ('chr1:182855:-', 1), ('chr1:1449141:+', 1), ('chr3:1722190:+', 1), ('chr6:3148935:-', 1), ('chr1:7250446:-', 1), ('chr1:5513126:+', 1), ('chr2:455150:+', 1), ('chr2:1866159:+', 1), ('chr3:291875:+', 1), ('chr1:3821780:-', 1), ('chr3:2107619:-', 1), ('chr2:4605968:-', 1), ('chr5:3773389:-', 1), ('chr1:10033758:-', 1), ('chr7:1096722:-', 1), ('chr2:1154399:-', 1), ('chr3:626243:+', 1), ('chr1:845857:-', 1)]"
TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,25,25,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24)]"
ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1575,1575,chr2,+,4661375,Single,1469,36,"[('chr2:4661375:+', 1469), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:4661374:+', 2), ('chr2:2025832:+', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr1:170029:-', 1), ('chr4:3545212:+', 1), ('c

In [17]:
#this takes a few minutes
def get_gene_by_bc_position(pos):
    
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold']==scaffold]
    scaffold_barcodes = df[df['scaffold']==scaffold]
    
    #divide scaffold_gff.shape by 2 here to get number of genes (original shape includes promoters)
    print('Mapping {} barcodes on {} to {} genes on {}'.format(scaffold_barcodes.shape[0], scaffold,
                                                              round(scaffold_gff.shape[0]/2), scaffold))

    #turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, 1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    #for each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)

Mapping 1619 barcodes on chr1 to 1356 genes on chr1
Mapping 926 barcodes on chr2 to 715 genes on chr2
Mapping 724 barcodes on chr3 to 605 genes on chr3
Mapping 491 barcodes on chr4 to 535 genes on chr4
Mapping 358 barcodes on chr5 to 492 genes on chr5
Mapping 299 barcodes on chr6 to 396 genes on chr6
Mapping 569 barcodes on chr7 to 455 genes on chr7


In [18]:
#check that a 'gene' column has been added to df

df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,gene
barcode,,,,,,,,,,,
AGGCTCAAAGTACCTGTTAA,TTAACAGGTACTTTGAGCCT,33,33,chr4,+,1861183,Single,28,2,"[('chr4:1861183:+', 28), ('chr5:3550691:+', 1), ('chr7:1456504:+', 1), ('chr5:3256020:-', 1)]",proteinId=2306381
CTAGGTTGGAAGAACACACT,AGTGTGTTCTTCCAACCTAG,2181,2181,chr2,-,1265125,Concat,1408,639,"[('chr2:1265125:-', 1408), ('chr2:1265122:-', 4), ('chr2:2592533:+', 4), ('chr4:2180439:-', 3), ('chr1:8295228:+', 2), ('chr1:761707:+', 2), ('chr1:212137:+', 2), ('chr1:4996653:+', 2), ('chr2:4327814:-', 2), ('chr2:2395934:-', 2), ('chr5:290815:+', 2), ('chr2:4314401:+', 2), ('chr2:1105155:+', 2), ('chr2:4661375:+', 2), ('chr2:1265123:-', 2), ('chr1:954628:+', 1), ('chr7:2785623:+', 1), ('chr6:1720330:-', 1), ('chr1:2266535:-', 1), ('chr1:7182043:+', 1), ('chr7:1448797:+', 1), ('chr2:4234596:+', 1), ('chr3:3398001:-', 1), ('chr5:4165320:-', 1), ('chr3:4885693:+', 1), ('chr7:1746287:+', 1), ('chr1:9216225:+', 1), ('chr2:1114508:+', 1), ('chr6:3039724:-', 1), ('chr3:4887708:-', 1), ('chr1:7436906:+', 1), ('chr1:10013646:+', 1), ('chr2:1537912:+', 1), ('chr2:561950:+', 1), ('chr4:1614007:-', 1), ('chr1:7748225:-', 1), ('chr2:2194574:-', 1), ('chr2:1553014:+', 1), ('chr6:2863705:-', 1), ('chr2:664147:+', 1), ('chr2:2256078:+', 1), ('chr1:2700577:-', 1), ('chr1:10103069:+', 1), ('chr1:6914244:-', 1), ('chr5:3062535:-', 1), ('chr1:2068221:+', 1), ('chr2:5250247:-', 1), ('chr2:5426773:+', 1), ('chr6:288713:+', 1), ('chr1:4328966:+', 1), ('chr6:612013:+', 1), ('chr6:4085274:+', 1), ('chr3:1944078:+', 1), ('chr2:1265129:-', 1), ('chr1:7972497:-', 1), ('chr2:1855143:+', 1), ('chr2:4291019:-', 1), ('chr2:2305298:+', 1), ('chr5:2709153:+', 1), ('chr2:1114472:+', 1), ('chr5:2602384:+', 1), ('chr6:2322131:-', 1), ('chr4:1625473:+', 1), ('chr2:2720688:-', 1), ('chr2:470383:-', 1), ('chr2:1393197:+', 1), ('chr7:3977511:-', 1), ('chr2:4258145:-', 1), ('chr3:1646949:+', 1), ('chr1:242927:-', 1), ('chr1:2295113:-', 1), ('chr1:6511163:-', 1), ('chr1:8696473:-', 1), ('chr1:6354388:-', 1), ('chr1:2643374:+', 1), ('chr1:8732382:-', 1), ('chr1:6886045:+', 1), ('chr7:40481:-', 1), ('chr4:144100:+', 1), ('chr6:72928:-', 1), ('chr5:2355634:+', 1), ('chr1:8031958:-', 1), ('chr1:212113:-', 1), ('chr2:1137525:+', 1), ('chr1:6138485:+', 1), ('chr2:4663687:-', 1), ('chr6:105262:-', 1), ('chr6:1914656:-', 1), ('chr7:2979024:-', 1), ('chr7:2372256:-', 1), ('chr2:1265085:+', 1), ('chr6:2608888:-', 1), ('chr2:291854:+', 1), ('chr7:2325040:-', 1), ('chr7:1120573:+', 1), ('chr3:2416675:+', 1), ('chr1:7836540:-', 1), ('chr3:1904761:+', 1), ('chr1:182855:-', 1), ('chr1:1449141:+', 1), ('chr3:1722190:+', 1), ('chr6:3148935:-', 1), ('chr1:7250446:-', 1), ('chr1:5513126:+', 1), ('chr2:455150:+', 1), ('chr2:1866159:+', 1), ('chr3:291875:+', 1), ('chr1:3821780:-', 1), ('chr3:2107619:-', 1), ('chr2:4605968:-', 1), ('chr5:3773389:-', 1), ('chr1:10033758:-', 1), ('chr7:1096722:-', 1), ('chr2:1154399:-', 1), ('chr3:626243:+', 1), ('chr1:845857:-', 1)]",
TAGCCTAATTCTCTTTTAAT,ATTAAAAGAGAATTAGGCTA,25,25,chr1,+,6115762,Single,24,1,"[('chr1:6115762:+', 24)]",proteinId=2297294
ATTACACGGGTCCATGATGA,TCATCATGGACCCGTGTAAT,1575,1575,chr2,+,4661375,Single,1469,36,"[('chr2:4661375:+', 1469), ('chr2:4661376:+', 5), ('chr2:2256078:+', 4), ('chr2:4327814:-', 2), ('chr1:987112:-', 2), ('chr3:2416675:+', 2), ('chr1:7974718:+', 2), ('chr2:4661377:+', 2), ('chr1:8833132:-', 2), ('chr2:4661374:+', 2), ('chr2:2025832:+', 2), ('chr2:470383:-', 1), ('chr2:1789278:+', 1), ('chr1:3866952:+', 1), ('chr1:3821780:-', 1), ('chr7:221711:+', 1), ('chr2:2194574:-', 1), ('chr3:2413694:+', 1), ('chr3:740765:-', 1), ('chr7:1196515:-', 1), ('chr4:3414371:-', 1), ('chr6:72928:-', 1), ('chr1:6511162:-', 1), ('chr2:4314401:+', 1), ('chr1:2266535:-', 1), ('chr4:425807:+', 1), ('chr2:2592533:+', 1), ('chr1:9004567:-', 1), ('chr1:8069094:+', 1), ('chr5:2084104:-', 1), ('chr7:3471884:-', 1), ('chr7:602092:+', 1), ('chr1:9216225:+', 1), ('chr

In [19]:
print('{} barcodes not in coding sequences'.format(df[df['gene']==''].shape[0]))
print()

print('{} barcodes in coding sequences'.format(df[df['gene']!=''].shape[0]))
print('{} barcodes in Kl coding sequences'.format(df[df['gene'].str.startswith('kl',na=False)].shape[0]))
print('{} barcodes in Km coding sequences'.format(df[df['gene'].str.startswith('km',na=False)].shape[0]))

3022 barcodes not in coding sequences

1964 barcodes in coding sequences
0 barcodes in Kl coding sequences
0 barcodes in Km coding sequences


In [20]:
df.to_csv(annotated_poolfile_path, sep='\t')

In [ ]:
gff[gff['ID']=='kmYHR034C']